In [1]:
import os

In [3]:
base = '../../selected/'
record = {}
for dn in os.listdir(base):
    record[dn] = len(os.listdir(os.path.join(base, dn)))

In [5]:
import pandas as pd

In [8]:
comp = pd.read_csv('../data/summary.txt', sep='\t', header=None)

In [9]:
comp.columns = ['case', 'num']

In [11]:
re_df = pd.DataFrame.from_dict(record, orient='index')

In [14]:
comp_df = re_df.merge(comp, left_index=True, right_on='case')

In [20]:
comp_df['gap'] = abs(comp_df.iloc[:, 0] - comp_df.iloc[:, 2]) / comp_df.iloc[:, 0]

In [21]:
comp_df.loc[comp_df['gap'] > 0.1]

,0,case,num,gap
161,209,TCGA-AC-A3W5-01Z-00-DX1,182,0.129187


In [44]:
a = random.choices(range(100), k=10)

In [49]:
from itertools import chain

In [55]:
def f1():
    n = 0
    while n < 10:
        yield n, n+1
        n += 1

In [56]:
def f2():
    n = 10
    while n < 20:
        yield n, n+1
        n += 1

In [60]:
if f1:
    print(1)

1


In [59]:
for (a,b),(c,d) in zip(f1(), f2()):
    print(a, c)

0 10
1 11
2 12
3 13
4 14
5 15
6 16
7 17
8 18
9 19


In [48]:
a

[6, 8, 34, 47, 54, 64, 67, 67, 72, 86]

In [32]:
a = [11]

In [37]:
a = [1]
b = [2]
c = [a, b]
for num in c:
    num.append(1)
print(c)


[[1, 1], [2, 1]]


In [63]:
import os
import pandas as pd
import random
import cv2
from keras.applications.nasnet import NASNetMobile, preprocess_input
random.seed(42)

Using TensorFlow backend.


In [27]:
def case2path(x_p):
    cur_dirs = os.listdir(x_p)
    result = {}
    for dir_n in cur_dirs:
        case = dir_n[:12]
        if case in result:
            continue
        result[case] = os.path.join(x_p, dir_n)
    return pd.DataFrame.from_dict(result, orient='index', columns=['path'])

In [ ]:
def cur_table_creator(selected_p, target_p='../data/Target.xlsx'):
    target = pd.read_excel(target_p)
    case_path_df = case2path(selected_p)
    match = case_path_df.merge(target, left_index=True, right_on='sample')
    print(len(match)/len(target))
    match.reset_index(drop=True, inplace=True)
    return match

In [71]:
def table_creator(cur_table, train_ratio=0.8):
    idx = cur_table.index.tolist()

    random.shuffle(idx)

    train_size = round(len(idx) * train_ratio)
    train_idx = idx[:train_size]
    test_idx = idx[train_size:]

    train_table = cur_table.iloc[train_idx]
    test_tabel = cur_table.iloc[test_idx]

    train_table.sort_values('duration', inplace=True)
    test_tabel.sort_values('duration', inplace=True)

In [78]:
def read_dir(dir_p):
    pool = os.listdir(dir_p)
    sel = random.choice(pool, 1)
    x = os.path.join(dir_p, sel)
    return preprocess_input(cv2.imread(x))

In [84]:
def gen_data(df_sort):
    while 1:
        n = 0
        x, T, E = [], [], []
        for item in df_sort.iterrows():
            path = item[1][0]
            dur = item[1][2]
            obs = item[1][3]
            x.append(read_dir(path))
            T.append(dur)
            E.append(obs)
            n += 1
            if n > 100:
                yield x, T, E
                x, T, E = [], [], []
                n = 0

In [85]:
test_gen = gen_data(test_tabel)

In [86]:
item = next(gen_data)

TypeError: 'function' object is not an iterator

In [ ]:
def data_flow():
    aug = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,)
    return aug

In [ ]:
def train():
    ada = Adagrad(lr=1e-3, decay=0.1)
    # rmsprop=RMSprop(lr=1e-5, rho=0.9, epsilon=1e-8)
    cheak_list = [EarlyStopping(monitor='loss', patience=10),
                ModelCheckpoint(filepath=os.path.join(dst, 'modelcc.h5')
                , monitor='loss', save_best_only=True),
                TensorBoard(log_dir=os.path.join(dst, 'cell_log'), 
                histogram_freq=0)]
    model.fit_generator(
        ,
        steps_per_epoch=1,
        epochs=100,
        callbacks=cheak_list,
        shuffle=False)

In [1]:
from keras.models import Model
from keras.preprocessing.image import img_to_array, ImageDataGenerator
from keras.layers import Input, GlobalMaxPooling2D, GlobalAveragePooling2D, Flatten, Concatenate, Dropout, Dense
from keras import backend as K
from keras.regularizers import l2
from keras.optimizers import Adagrad, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.applications.nasnet import NASNetMobile, preprocess_input

In [2]:
inputs = Input((96, 96, 3))
base_model = NASNetMobile(include_top=False, input_shape=(96, 96, 3))#, weights=None
x = base_model(inputs)
out1 = GlobalMaxPooling2D()(x)
out2 = GlobalAveragePooling2D()(x)
out3 = Flatten()(x)
out = Concatenate(axis=-1)([out1, out2, out3])
out = Dropout(0.5)(out)
out = Dense(1, activation="linear", kernel_initializer='glorot_uniform', 
kernel_regularizer=l2(0.01), activity_regularizer=l2(0.01))(out)
model = Model(inputs, out)

In [3]:
model.load_weights('../model.h5', by_name='NASNet')

model.layers[1].trainable = False

model.summary()

In [6]:
def negative_log_likelihood(E):
	def loss(y_true,y_pred):
		hazard_ratio = K.exp(y_pred)
		log_risk = K.log(K.cumsum(hazard_ratio))
		uncensored_likelihood = K.transpose(y_pred) - log_risk
		censored_likelihood = uncensored_likelihood * E
		num_observed_event = K.sum([float(e) for e in E]) + 1
		return K.sum(censored_likelihood) / num_observed_event * (-1)
	return loss

In [ ]:
preprocess_input(cv2.imread(x))